# Project 1 – Decision Trees and Random Forests

#### Imports
Imports og random seed

In [ ]:


import numpy as np

from itertools import product
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from decision_tree import DecisionTree
from random_forest import RandomForest

# Reproducibility
np.random.seed(21)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load dataset

Laster inn letters.csv og deler opp i features (X) og labels (y), og deler inn i test/train datasett (80/20)

In [12]:
data = np.genfromtxt("letters.csv", delimiter=",", dtype=float, names=True)

feature_names = list(data.dtype.names[:-1])
target_name = data.dtype.names[-1]

X = np.array([data[f] for f in feature_names]).T
y = data[target_name].astype(int)

# 80/20 train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=0, shuffle=True, stratify=y
)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


X_train shape: (1600, 16), y_train shape: (1600,)
X_test shape: (400, 16), y_test shape: (400,)


### Hyperparametere

Setter opp verdier som skal testes i grid search for DecisionTree og RandomForest.

In [13]:
dt_params = {
    "criterion": ["entropy", "gini"],
    "max_depth": [None, 5, 10, 20],
    "max_features": [None, "sqrt", "log2"],
}

rf_params = {
    "n_estimators": [10, 20, 40],
    "max_depth": [5, 10, None],
    "criterion": ["entropy", "gini"],
    "max_features": ["sqrt", "log2"],
}


### Cross-validation

Funksjon for k-fold cross validation som regner ut accuracy.

In [14]:
def cross_val_score_custom(model_class, params, X, y, k=5, seed=0):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
    scores = []
    for train_idx, val_idx in skf.split(X, y):
        X_tr, X_val = X[train_idx], X[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]

        model = model_class(**params)
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_val)
        scores.append(accuracy_score(y_val, y_pred))
    return float(np.mean(scores))


### DecisionTree grid search

Tester kombinasjoner av hyperparametere og finner beste for DecisionTree.

In [15]:
best_dt_params = None
best_dt_score = -1.0

for criterion, max_depth, max_features in product(
    dt_params["criterion"], dt_params["max_depth"], dt_params["max_features"]
):
    params = {
        "criterion": criterion,
        "max_depth": max_depth,
        "max_features": max_features,
    }
    score = cross_val_score_custom(DecisionTree, params, X_train, y_train, k=5, seed=0)
    if score > best_dt_score:
        best_dt_score = score
        best_dt_params = params

print("Best DecisionTree params:", best_dt_params)
print("Best DecisionTree 5-fold CV accuracy:", round(best_dt_score, 4))


Best DecisionTree params: {'criterion': 'gini', 'max_depth': 20, 'max_features': None}
Best DecisionTree 5-fold CV accuracy: 0.8975


### RandomForset - grid Serach
Tester kombinasjoner av hyperparametere og finner beste for RandomForest.

In [16]:
best_rf_params = None
best_rf_score = -1.0

for n_estimators, max_depth, criterion, max_features in product(
    rf_params["n_estimators"],
    rf_params["max_depth"],
    rf_params["criterion"],
    rf_params["max_features"],
):
    params = {
        "n_estimators": n_estimators,
        "max_depth": max_depth,
        "criterion": criterion,
        "max_features": max_features,
    }
    score = cross_val_score_custom(RandomForest, params, X_train, y_train, k=5, seed=0)
    if score > best_rf_score:
        best_rf_score = score
        best_rf_params = params

print("Best RandomForest params:", best_rf_params)
print("Best RandomForest 5-fold CV accuracy:", round(best_rf_score, 4))


Best RandomForest params: {'n_estimators': 40, 'max_depth': None, 'criterion': 'entropy', 'max_features': 'sqrt'}
Best RandomForest 5-fold CV accuracy: 0.965


### Final evaluation

We retrain our models with the best hyperparameters on the full training set and evaluate on the test set.


In [17]:
# DecisionTree
dt_best = DecisionTree(**best_dt_params)
dt_best.fit(X_train, y_train)
dt_test_acc = accuracy_score(y_test, dt_best.predict(X_test))

# RandomForest
rf_best = RandomForest(**best_rf_params)
rf_best.fit(X_train, y_train)
rf_test_acc = accuracy_score(y_test, rf_best.predict(X_test))

print("Custom DecisionTree test accuracy:", dt_test_acc)
print("Custom RandomForest test accuracy:", rf_test_acc)


Custom DecisionTree test accuracy: 0.895
Custom RandomForest test accuracy: 0.9775


### Trene beste modeller 
Trener beste modeler

In [18]:
# DecisionTree (custom)
dt_best = DecisionTree(**best_dt_params)
dt_best.fit(X_train, y_train)
dt_test_pred = dt_best.predict(X_test)
dt_test_acc = accuracy_score(y_test, dt_test_pred)

# RandomForest (custom)
rf_best = RandomForest(**best_rf_params)
rf_best.fit(X_train, y_train)
rf_test_pred = rf_best.predict(X_test)
rf_test_acc = accuracy_score(y_test, rf_test_pred)

print("Custom DecisionTree test accuracy:", round(dt_test_acc, 4))
print("Custom RandomForest test accuracy:", round(rf_test_acc, 4))


Custom DecisionTree test accuracy: 0.9
Custom RandomForest test accuracy: 0.9825


### SKLearn modeller

Trener modell med sklearn med de samme paramterene

In [19]:
# Sklearn DecisionTree
sk_dt = DecisionTreeClassifier(
    criterion=best_dt_params["criterion"],
    max_depth=best_dt_params["max_depth"],
    max_features=best_dt_params["max_features"],
    random_state=0,
)
sk_dt.fit(X_train, y_train)
sk_dt_acc = accuracy_score(y_test, sk_dt.predict(X_test))

# Sklearn RandomForest
sk_rf = RandomForestClassifier(
    n_estimators=best_rf_params["n_estimators"],
    max_depth=best_rf_params["max_depth"],
    criterion=best_rf_params["criterion"],
    max_features=best_rf_params["max_features"],
    random_state=0,
)
sk_rf.fit(X_train, y_train)
sk_rf_acc = accuracy_score(y_test, sk_rf.predict(X_test))

print("Sklearn DecisionTree test accuracy:", round(sk_dt_acc, 4))
print("Sklearn RandomForest test accuracy:", round(sk_rf_acc, 4))


Sklearn DecisionTree test accuracy: 0.94
Sklearn RandomForest test accuracy: 0.985


### Rapport 
Detaljert evaluering

In [20]:
def report(name, y_true, y_pred):
    print(f"\n=== {name} ===")
    print(classification_report(y_true, y_pred, digits=4))
    print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))

report("Custom DecisionTree", y_test, dt_test_pred)
report("Custom RandomForest", y_test, rf_test_pred)
report("Sklearn DecisionTree", y_test, sk_dt.predict(X_test))
report("Sklearn RandomForest", y_test, sk_rf.predict(X_test))



=== Custom DecisionTree ===
              precision    recall  f1-score   support

           0     0.9500    0.8906    0.9194        64
           1     0.8219    0.8696    0.8451        69
           2     0.9688    0.9538    0.9612        65
           3     0.8806    0.8806    0.8806        67
           4     0.9344    0.8906    0.9120        64
           5     0.8667    0.9155    0.8904        71

    accuracy                         0.9000       400
   macro avg     0.9037    0.9001    0.9014       400
weighted avg     0.9020    0.9000    0.9005       400

Confusion matrix:
 [[57  0  2  2  0  3]
 [ 0 60  0  3  4  2]
 [ 0  2 62  0  0  1]
 [ 0  6  0 59  0  2]
 [ 3  1  0  1 57  2]
 [ 0  4  0  2  0 65]]

=== Custom RandomForest ===
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000        64
           1     0.9583    1.0000    0.9787        69
           2     0.9692    0.9692    0.9692        65
           3     1.0000    1.0000   